In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(0)
np.random.seed(0)

plt.rcParams['figure.figsize'] = (10, 10)

## Initialisation des variables

$$f(x)=\frac{1}{4\pi}\exp \left \{ -\frac{1}{2} \left (x-\begin{pmatrix}2\\ 2\end{pmatrix} \right)^T\begin{bmatrix}2 & 0\\ 0 & 2\end{bmatrix}^{-1}\left (x-\begin{pmatrix}2\\ 2\end{pmatrix} \right ) \right \}$$
$$f(x)=\frac{1}{12\pi}\exp\left \{ -\frac{1}{2}\left(x-\begin{pmatrix}-4\\ -4\end{pmatrix} \right)^T\begin{bmatrix}6 & 0\\ 0 & 6\end{bmatrix}^{-1}\left (x-\begin{pmatrix}-4\\ -4\end{pmatrix}\right ) \right \}$$

In [ ]:
# Simulation de n = 256 données X=(X1, X2) suivant une loi normale bidimensionnelle
# de moyenne (0,0), de la variance Var(X1) = 1, Var(X2) = 1 et la covariance Cov(X1, X2) = 0.5


mean_1 = np.array([2, 2])
mean_2 = np.array([-4, -4])

cov_1 = np.array([[2, 0], [0, 2]])
cov_2 = np.array([[6, 0], [0, 6]])

X1 = (np.random.multivariate_normal(mean_1, cov_1, 128))
X2 = (np.random.multivariate_normal(mean_2, cov_2, 128))


# Affichage des données

plt.plot(X1[:, 0], X1[:,1], "o", label = 'Individu', color = "black")
plt.plot(X2[:, 0], X2[:,1], "o", label = 'Individu', color = "red")


## 1.1.2.1 Test de la Méthode de `K-means`

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from scipy.spatial import Voronoi, voronoi_plot_2d

clusters = 5

kmeans = KMeans(n_clusters=clusters, n_init=1, init='k-means++')

data = np.concatenate((X1, X2), axis=0)
kmeans.fit(data)

y_kmeans = kmeans.predict(data)

centers = kmeans.cluster_centers_

# Affichage d'un diagramme de Voronoi à partir des clusters. Hors TP
# if clusters > 2:
#     bounds = Voronoi(centers)
#     voronoi_plot_2d(bounds)

plt.scatter(data[:, 0], data[:, 1], c=y_kmeans)
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

plt.show()
print('Taux d\'erreur : ' + str(adjusted_rand_score(kmeans.labels_, y_kmeans)))
if (adjusted_rand_score(kmeans.labels_, y_kmeans) == 1.0):
    print("Les clusters sont identiques")



La variable `kmeans.labels_` correspond à la liste des clusters associés à chaque point. Par exemple, si `kmeans.labels_[0] == 1`, cela signifie que le points n°0 appartient au cluster n°1. On note que si `K` le nombre de clusters demandé, alors `0 <= kmeans.labels_[x] < K`.

Avec `K = 5`, nous avons le résultat affiché au dessus.

D'après la documentation, si `adjusted_rand_score() == 1`, alors les deux groupes de clusters comparés sont identiques (à une permutation près).

Le paramètre `n_init` donne le nombre d'itérations de calcul à faire (en partant de points aléatoires différents à chaque fois), avant de choisir le meilleur résultat. Dans notre cas, nous n'avons pas remarqué de résultats significativement différents, cependent le temps de calcul a été évidemment `n_init` fois plus important.



## 1.1.2.2 Choix du bon nombre de cluster en utilisant les silhouettes

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

def plot_inertia(data_kmeans):
    clusters_max = 15
    s_coefs = []
    s_intertia = []
    for i in range(2, clusters_max) :
        kmeans = KMeans(n_clusters=i, n_init=1, init='k-means++').fit(data_kmeans)
        labels = kmeans.labels_

        s_avg = silhouette_score(data_kmeans, labels)
        print(str(i) + " clusters, coef de silhouette : " + str(s_avg) + ", intertie : " + str(kmeans.inertia_))
        s_coefs.append(s_avg)
        s_intertia.append(kmeans.inertia_)
        # print("Sample silhouete value : " + str(silhouette_samples(data, labels)))

    fig, ax1 = plt.subplots()
    ax1.set_xlabel("nb clusters")
    ax1.set_ylabel("coef de silhouette")
    ax1.plot(range(2, clusters_max), s_coefs, color="blue")
    ax2 = ax1.twinx()
    ax2.set_ylabel("inertia", color="red")
    ax2.plot(range(2, clusters_max), s_intertia, color="red")
    ax2.tick_params(axis='y', labelcolor="red")
    fig.tight_layout()
    plt.show()

plot_inertia(data)

Le nombre de clusters idéal est déterminé par un coude sur le graphique de coefficients de silhouette. D'apres le graphique ci-dessus, nous pouvons distinguer un coude pour `K = 6`, le nombre de clusters idéal est donc 6.

## 1.1.3 Clustering Ascendant Hiérarchique (CAH)

In [ ]:
from scipy.cluster.hierarchy import linkage as CAH, dendrogram, fcluster

clusters = 3

def plot_z_complete(method):
    plt.gca().set_title(method)
    Z_complete = CAH(data, method=method, metric='euclidean')

    threshold_value = Z_complete[Z_complete.shape[0] - clusters, 2]

    plt.axhline(y=threshold_value, c='grey', lw=1, linestyle='dashed')
    threshold_value_before = Z_complete[Z_complete.shape[0] - clusters + 1, 2]
    dendrogram(Z_complete,  color_threshold=threshold_value_before)
    groupes_cah = fcluster(Z_complete, t=threshold_value, criterion="distance")
    print("Méthode : " + method + "\t" + str(adjusted_rand_score(groupes_cah, y_kmeans)))

plt.title("CAH")

plt.subplot(2, 2, 1)
plot_z_complete('ward')

plt.subplot(2, 2, 2)
plot_z_complete('complete')

plt.subplot(2, 2, 3)
plot_z_complete('single')

plt.subplot(2, 2, 4)
plot_z_complete('average')

In [ ]:
clusters = 3

kmeans = KMeans(n_clusters=clusters, n_init=1, init='k-means++')
kmeans.fit(data)
y_kmeans = kmeans.predict(data)

def compare_z_complete(method):
    Z_complete = CAH(data, method=method, metric='euclidean')
    threshold_value = Z_complete[Z_complete.shape[0] - clusters, 2]
    groupes_cah = fcluster(Z_complete, t=threshold_value, criterion="distance")
    print("Méthode : " + method + "\t" + str(adjusted_rand_score(groupes_cah, y_kmeans)))
    
compare_z_complete('ward')
compare_z_complete('complete')
compare_z_complete('single')
compare_z_complete('average')
    

Pour `K = 2`, n remarque aussi que les clusters formés par les méthodes `ward`, `complete` et `average` sont très proches des résultats obtenus avec `K-Means` (adjusted_rand_score > 0.9). Cependant la méthode `single` donne des résultats très différents (adjusted_rand_score < 0.15).

## Utilisation de la méthode K-Means pour la réduction de couleur


In [ ]:
import matplotlib.image as mpimg
plt.rcParams['figure.figsize'] = (15, 15)

# Affiche l'image après réduction à i couleurs (via clusters) 
def image_kmeans(i):
    plt.title(str(i) + " clusters")
    
    raw_img = mpimg.imread('visage.bmp')
    img = np.float32(raw_img)
    a = np.reshape(img, (img.shape[0]*img.shape[1],3))
    
    kmeans = KMeans(n_clusters=i,n_init=1,init='k-means++')
    kmeans.fit(a)

    for i in range(a.shape[0]):
        a[i] = kmeans.cluster_centers_[kmeans.labels_[i],:]
    
    res_img = np.reshape(a, (img.shape[0], img.shape[1], 3))
    plt.imshow(res_img/255)

In [ ]:
K = [5, 15, 30, 50, 75, 100, 150, 200]


for index, k in enumerate(K):
    plt.subplot(3, 3, index + 1)
    image_kmeans(k)
    
plt.subplot(3, 3, 9)
plt.imshow(mpimg.imread('visage.bmp'))
plt.title("image initiale")


On constate que le bleu est minime par rapport au rouge et au vert, il a donc tendance à rentrer dans les clusters de ceux-ci, ce qui explique la couleur plus orangé des yeux sur l'image traité avec le kmeans.

## Clustering de données de températures 

In [ ]:
import pandas as pd

clusters = 7

data_temperature = pd.read_csv("temperatures.csv",sep=";",decimal=".",header=0,index_col = 0)
# n = len(data_temperature)
plt.subplot(2, 2, 1)
plt.title("Janvier -> Décembre")
data = data_temperature.drop(columns=['Region', 'Moyenne', 'Amplitude', 'Latitude', 'Longitude'])
Z_complete = CAH(data, method='average', metric='euclidean')

threshold_value = Z_complete[Z_complete.shape[0] - clusters, 2]
plt.axhline(y=threshold_value, c='grey', lw=1, linestyle='dashed')
city_name = list(data.index)
a = dendrogram(Z_complete, labels=city_name, color_threshold=threshold_value)
plt.subplot(2, 2, 2)
coord = data_temperature.loc[:, ['Latitude', 'Longitude']].values
#Cette ligne permet d’extraire les coordonnees
groupes_cah_1 = fcluster(Z_complete, t=threshold_value, criterion="distance")
plt.scatter(coord[:, 1], coord[:, 0], cmap='viridis', c=groupes_cah_1, marker="o", s=200)
#On place les points
for i, txt in enumerate(city_name):
    plt.annotate(txt, (coord[i, 1], coord[i, 0])) #On place le nom des villes
    
plt.subplot(2, 2, 3)
plt.title("Juillet -> Décembre, Moy, Amplitude, Latitude et Longitude")
data = data_temperature.drop(columns=['Janvier', 'Fevrier', 'Mars', 'Avril', 'Mai', 'Juin', 'Region'])
Z_complete = CAH(data, method='average', metric='euclidean')
threshold_value = Z_complete[Z_complete.shape[0] - clusters, 2]
plt.axhline(y=threshold_value, c='grey', lw=1, linestyle='dashed')
city_name = list(data.index)
a = dendrogram(Z_complete, labels=city_name, color_threshold=threshold_value)
plt.subplot(2, 2, 4)
coord = data_temperature.loc[:, ['Latitude', 'Longitude']].values
#Cette ligne permet d’extraire les coordonnees
groupes_cah_2 = fcluster(Z_complete, t=threshold_value, criterion="distance")
plt.scatter(coord[:, 1], coord[:, 0], cmap='viridis', c=groupes_cah_2, marker="o", s=200)
#On place les points
for i, txt in enumerate(city_name):
    plt.annotate(txt, (coord[i, 1], coord[i, 0])) #On place le nom des villes

In [ ]:
nb_clusters_cities = 7

kmeans = KMeans(n_clusters=nb_clusters_cities, n_init=10, init='k-means++').fit(coord)
y_kmeans = kmeans.predict(coord)

plt.scatter(coord[:, 1], coord[:, 0], c=y_kmeans, marker="o", s=200)

for i, txt in enumerate(city_name):
    plt.annotate(txt, (coord[i, 1], coord[i, 0])) #On place le nom des villes

In [ ]:
plot_inertia(coord)

Avec la règle du coude, on peut estimer la valeur optimale de `K = 7`.

In [ ]:
print("Groupe 1 : " + str(adjusted_rand_score(groupes_cah_1, y_kmeans)))
print("Groupe 2 : " + str(adjusted_rand_score(groupes_cah_2, y_kmeans)))

On remarque les clusters formés par le groupe de donnés 1 (Janvier -> Décembre) sont assez différents de ceux formés par kmeans, aussi, les clusters formés par le groupe de données 2 (Juillet -> Décembre, Moyenne, Amplitude, Latitude et Longitude) est un peu plus proche du résultat de kmeans que le groupe 1 même s'il reste peu similaire.